In [9]:
import random
import pandas as pd
from sage.graphs.graph_generators import graphs
import numpy as np
import pandas as pd


#####################################################################

def SDCTD_stevilo2(g):
    h = g.complement()

    p = MixedIntegerLinearProgram(maximization = False)
    x = p.new_variable(binary = True)
    p.set_objective(sum([x[v] for v in g]))

    for v in g.vertices():
        neighbors = g.neighbors(v)  
        if neighbors:
            p.add_constraint(sum((x[w] + x[v]) for w in neighbors) >= 1)
        else:
            p.add_constraint(x[v] == 1)
            
    zaustavi = False

    for v in g.vertices():
        neighbors_complement = h.neighbors(v)  
        if neighbors_complement:  
            p.add_constraint(sum(x[w] for w in neighbors_complement) >= 1)

        else: 
            zaustavi = True
            break 

    if not zaustavi == True:
        p.solve()
        x = p.get_values(x)
        return(sum(1 for i in x.values() if i == 1))
    else:
        return None

#####################################################################
grafi = []  

for i in range(2, 9):
    for G in graphs.nauty_geng(f'{i} -c'):  # 'c' pomeni povezane grafe
        grafi.append(G)

tabela = []

# Izračunamo minimalno in maksimalno stopnjo za vsak graf
for G in grafi:
    min_stopnja = min(G.degree())  
    max_stopnja = max(G.degree()) 

    dominacijsko_stevilo = SDCTD_stevilo2(G)
   
    n = G.order()  # Število vozlišč grafa   
    
    if dominacijsko_stevilo is None:
        dominacijsko_stevilo = 0
    
##########################################################################

    E1 = dominacijsko_stevilo >= n / max_stopnja

##########################################################################

    if n >= 3 and max_stopnja <= (n-2):
        E2 = dominacijsko_stevilo >= n - max_stopnja
    else:
        E2 = ''
        
##########################################################################

    E3 = dominacijsko_stevilo <= ((1 + np.log(min_stopnja)) / (min_stopnja)) * n

##########################################################################
   
    if min_stopnja == 1:
        E4 = dominacijsko_stevilo <= (n / 2)
    else:
        E4 = ''
        
##########################################################################
   
    if min_stopnja == 2:
        E5 = dominacijsko_stevilo <= (2 * n) / 5
    else:
        E5 = ''
        
##########################################################################
   
    if min_stopnja == 3:
        E6 = dominacijsko_stevilo <= (3 * n) / 8
    else:
        E6 = ''
        
##########################################################################
   
    if min_stopnja == 4:
        E7 = dominacijsko_stevilo <= (4 * n) / 11
    else:
        E7 = ''
        
##########################################################################
   
    if min_stopnja == 5:
        E8 = dominacijsko_stevilo <= (5 * n) / 14
    else:
        E8 = ''

##########################################################################

    tabela.append([n, min_stopnja, max_stopnja, dominacijsko_stevilo, E1, E2, E3, E4, E5, E6, E7, E8])


tabela_grafov = pd.DataFrame(tabela, columns=['Število vozlišč', 'Min stopnja', 'Max stopnja', 'Dominacijsko število', 'E1', 'E2', 'E3', 'E4', 'E5', 'E6', 'E7', 'E8'])


false_counts = {}
true_counts = {}

# Iteriramo čez vse stolpce E1-E8
for i in range(1, 9):
    col = f'E{i}'
    false_counts[col] = (tabela_grafov[col] == False).sum()
    true_counts[col] = (tabela_grafov[col] == True).sum()

# Izpišemo rezultate
for i in range(1, 9):
    col = f'E{i}'
    false = false_counts[col]
    true = true_counts[col]
    print(f"Število False v stolpcu {col}: {false}, delež: {false} / ({true} + {false})")
    
    
print(tabela_grafov)


Število False v stolpcu E1: 1341, delež: 1341 / (10771 + 1341)
Število False v stolpcu E2: 3203, delež: 3203 / (7657 + 3203)
Število False v stolpcu E3: 39, delež: 39 / (12073 + 39)
Število False v stolpcu E4: 9, delež: 9 / (4078 + 9)
Število False v stolpcu E5: 415, delež: 415 / (4848 + 415)
Število False v stolpcu E6: 396, delež: 396 / (1908 + 396)
Število False v stolpcu E7: 268, delež: 268 / (139 + 268)
Število False v stolpcu E8: 16, delež: 16 / (29 + 16)
       Število vozlišč  Min stopnja  Max stopnja Dominacijsko število     E1  \
0                    2            1            1                    0  False   
1                    3            1            2                    0  False   
2                    3            2            2                    0  False   
3                    4            1            3                    0  False   
4                    4            1            2                    2   True   
...                ...          ...          ...       